In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import mean_absolute_error 
from rgf.rgf import RGFRegressor

/home/y.romanikhin/.virtualenvs/venv/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("train_big.csv")
test = pd.read_csv("test_big.csv")

In [3]:
shift = 200

In [33]:
ids_train = train["id"]
ids_test = test["id"]

y_train = np.log(train['loss'] + shift)
X_train = train.drop(['loss','id'], axis=1)
X_test = test.drop(['loss','id'], axis=1)

In [ ]:
## create holdout
X_train_local, X_val, y_train_local, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [84]:
kf = KFold(X_train.shape[0], n_folds = nfolds, shuffle = True, random_state = 111)


In [85]:
from sklearn.ensemble import ExtraTreesRegressor

clf = ExtraTreesRegressor(n_estimators=2, criterion='mse', max_depth=2,)

In [101]:
%%time

oof_df = pd.DataFrame()

for i, (itr, ite) in enumerate(kf):
    x_tr = X_train.loc[itr]
    y_tr = y_train.loc[itr].values
    ind_tr = ids_train.loc[itr].values
    
    x_te = X_train.loc[ite]
    y_te = y_train.loc[ite].values
    ind_te = ids_train.loc[ite].values
    
    clf.fit(x_tr, y_tr)
    pred_oof = clf.predict(x_te)
    
    oof_df = oof_df.append(pd.DataFrame(np.vstack([ind_te, y_te, pred_oof]).T, 
                               columns=["id", "y_true", "y_pred"]).set_index("id"))

oof_df = oof_df.sort_index()    
oof_df.to_csv("oof_preds/oof_df.csv")

In [80]:
clf.fit(x_tr, y_tr)
pred_oof = clf.predict(x_te)

In [81]:
pd.DataFrame(np.vstack([ind_te, y_te, pred_oof]).T, columns=["id", "y_true", "y_pred"]).set_index("id")

,y_true,y_pred
id,,
469729.0,6.971434,7.529062
469733.0,7.489423,8.068243
469734.0,7.516743,7.529062
469736.0,8.620825,8.173037
469737.0,8.160398,7.529062
469738.0,7.027128,7.529062
469744.0,8.869609,7.529062
469746.0,7.899139,8.173037
469747.0,8.176313,7.529062


In [78]:
ids_train.loc[ite].values

array([469729, 469733, 469734, ..., 587630, 587632, 587633])

In [56]:
pred_oof

array([ 8.16977231,  7.79475991,  7.51817222, ...,  7.51817222,
        7.51817222,  7.51817222])

In [59]:
x_te.shape

(37663, 725)

In [ ]:
def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [19]:
%%time

rgf = RGFRegressor(max_leaf=1000,
                   algorithm="RGF",
                   test_interval=100, loss='LS', l2=0.005, sl2=None, reg_depth=1, verbose=1)

rgf.fit(X_train_local, y_train_local)

"train": 
   algorithm=RGF
   train_x_fn=/tmp/rgf/train.data.x
   train_y_fn=/tmp/rgf/train.data.y
   Log:ON
   model_fn_prefix=/tmp/rgf/model
--------------------
Mon Dec  5 16:47:03 2016: Reading training data ... 
Mon Dec  5 16:47:38 2016: Start ... #train=150654
--------------------
Forest-level: 
   loss=LS
   max_leaf_forest=1000
   max_tree=500
   opt_interval=100
   test_interval=100
   num_tree_search=1
   Verbose:ON
   memory_policy=Generous
-------------
Training data: 725x150654, nonzero_ratio=1; managed as dense data.
-------------
Optimization: 
   loss=LS
   num_iteration_opt=10
   reg_L2=0.005
   opt_stepsize=0.5
   NormalizeTarget:ON
Tree-level: min_pop=10
Node split: reg_L2=0.005
--------------------
Mon Dec  5 16:51:13 2016: Calling optimizer with 33 trees and 100 leaves
Mon Dec  5 16:51:13 2016: Writing model: seq#=1
Mon Dec  5 16:54:10 2016: Calling optimizer with 52 trees and 200 leaves
Mon Dec  5 16:54:11 2016: Writing model: seq#=2
Mon Dec  5 16:56:42 2016: Call

In [24]:
pred_rgf = np.exp(rgf.predict(X_val)) - shift

Exception: Model learning result is not found @/tmp/rgf. This is rgf_python error.

In [21]:
score = mean_absolute_error(pred_rgf, np.exp(y_val) - shift)
score

1157.4393328587919

RGF/l2=0.01/sl2=None/reg_depth=1 - 1155.4236917994822

RGF_Opt/l2=0.01/sl2=None/reg_depth=1 - 1158.6360264446539

RGF_Sib/l2=0.01/sl2=None/reg_depth=1 - 1156.6491303637745

RGF/l2=0.01/sl2=None/reg_depth=1 - 1155.4236917994822

RGF/l2=0.05/sl2=None/reg_depth=1 - 1158.5445173799694

RGF/l2=0.005/sl2=None/reg_depth=1 - 1157.4393328587919

In [23]:
test.loc[:, 'loss'] = pred_rgf
#test.reset_index()[['id', 'loss']].to_csv('submissions/xgboost_basic_v1.csv', index = False)
#!gzip -f submissions/xgboost_basic_v1.csv

ValueError: Must have equal len keys and value when setting with an iterable